In [1]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
from keras.models import load_model
import itertools
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
train_path = './dataset/train'
test_path = './dataset/test'
valid_path = './dataset/valid'
classes = ['Alejandro_Toledo', 'Ariel_Sharon', 'Arnold_Schwarzenegger', 'Colin_Powell', 'Donald_Rumsfeld', 'George_W_Bush', 'Gerhard_Schroeder', 'Gloria_Macapagal_Arroyo', 'Hugo_Chavez', 'Jacques_Chirac', 'Jean_Chretien', 'Jennifer_Capriati', 'John_Ashcroft', 'Junichiro_Koizumi', 'Laura_Bush', 'Lleyton_Hewitt', 'Luiz_Inacio_Lula_da_Silva', 'Serena_Williams', 'Tony_Blair', 'Vladimir_Putin']

In [3]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=classes, batch_size = 10)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224), classes=classes, batch_size = 4)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=classes, batch_size = 10)

Found 1426 images belonging to 20 classes.
Found 240 images belonging to 20 classes.
Found 240 images belonging to 20 classes.


In [5]:
vgg16_model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)

In [6]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
model = Sequential()
for layer in vgg16_model.layers[:-2]:
    model.add(layer)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [8]:
for layer in model.layers:
    layer.trainable = False

In [9]:
model.add(Dense(4096, activation='relu'))
model.add(Dense(20, activation='softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [11]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit_generator(train_batches, steps_per_epoch=20, validation_data=valid_batches, validation_steps=8, epochs=10, verbose=2)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
20/20 - 14s - loss: 31.8939 - accuracy: 0.2950 - val_loss: 19.1466 - val_accuracy: 0.1562
Epoch 2/10
20/20 - 14s - loss: 6.3206 - accuracy: 0.4750 - val_loss: 6.7633 - val_accuracy: 0.2188
Epoch 3/10
20/20 - 14s - loss: 2.5312 - accuracy: 0.5050 - val_loss: 4.3754 - val_accuracy: 0.3438
Epoch 4/10
20/20 - 14s - loss: 1.5317 - accuracy: 0.6250 - val_loss: 2.0714 - val_accuracy: 0.3750
Epoch 5/10
20/20 - 14s - loss: 1.1230 - accuracy: 0.6900 - val_loss: 2.8465 - val_accuracy: 0.2812
Epoch 6/10
20/20 - 15s - loss: 0.9206 - accuracy: 0.7700 - val_loss: 2.5865 - val_accuracy: 0.5312
Epoch 7/10
20/20 - 15s - loss: 0.7669 - accuracy: 0.7750 - val_loss: 2.9264 - val_accuracy: 0.4062
Epoch 8/10
20/20 - 15s - loss: 0.5850 - accuracy: 0.8400 - val_loss: 2.0914 - val_accuracy: 0.5625
Epoch 9/10
20/20 - 15s - loss: 0.6480 - accuracy: 0.8367 - val_loss: 1.3807 - val_accuracy: 0.5625
Epoch 10/10
20/20 - 15s - loss:

In [13]:
model.save("test.h5")